In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [80]:
train_ind = pd.read_csv('/Users/jackconnors/Downloads/ind_train.csv')
train_dep = pd.read_csv('/Users/jackconnors/Downloads/dep_train.csv')
test_ind = pd.read_csv('/Users/jackconnors/Downloads/test_set_ind.csv')

In [6]:
train_dep.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [11]:
merge_train = pd.merge(train_ind, train_dep, on="id")

# Clean Data

In [16]:
merge_train.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

# Convert Date to Date Type and Create Duration Variable

In [17]:
merge_train['date_recorded'] = pd.to_datetime(merge_train['date_recorded'])
merge_train['construction_year'] = pd.to_datetime(merge_train['construction_year'])

In [18]:
merge_train['duration'] = merge_train['date_recorded'] - merge_train['construction_year']

In [26]:
resp_vars = merge_train[['duration', 'water_quality', 'management', 'id', 'status_group']]

In [27]:
resp_vars.head()

,duration,water_quality,management,id,status_group
0,15046 days 23:59:59.999998001,soft,vwc,69572,functional
1,15769 days 23:59:59.999997990,soft,wug,8776,functional
2,15760 days 23:59:59.999997991,soft,vwc,34310,functional
3,15732 days 23:59:59.999998014,soft,vwc,67743,non functional
4,15168 days 00:00:00,soft,other,19728,functional


# Classification Model

In [28]:
resp_vars.isna().sum()

duration         0
water_quality    0
management       0
id               0
status_group     0
dtype: int64

# Classification Model

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [60]:
x = resp_vars[['duration', 'water_quality', 'management']]
y= resp_vars[['status_group']]

In [62]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.values.ravel())

In [71]:
encoder = OneHotEncoder()

categorical_columns = ['water_quality', 'management']
encoded_features = encoder.fit_transform(x[categorical_columns])

encoded_df = pd.DataFrame(encoded_features.toarray(), columns=encoder.categories_[0].tolist() + encoder.categories_[1].tolist())

x_encoded = pd.concat([x.drop(categorical_columns, axis=1), encoded_df], axis=1)



In [63]:
x['duration_seconds'] = x['duration'].dt.total_seconds()

/var/folders/78/lyg5vs3x05n9nf7j9ttzb0yc0000gn/T/ipykernel_73236/2873239153.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['duration_seconds'] = x['duration'].dt.total_seconds()


In [65]:
x = x.drop('duration', axis=1)

array([0, 0, 0, ..., 0, 0, 0])

In [74]:
clf = RandomForestClassifier()

In [76]:
clf.fit(x_encoded, y_encoded)

RandomForestClassifier()

In [77]:
print(clf.feature_importances_)

[7.19082978e-01 3.90364583e-03 4.49150188e-03 5.50108890e-04
 3.97361791e-03 1.36060755e-02 5.24476550e-03 4.04518039e-02
 7.00384118e-02 1.24231896e-02 3.25936659e-03 3.42635823e-03
 4.57339658e-03 1.36374299e-02 8.79927375e-04 5.77087203e-03
 4.29723006e-02 6.70847030e-03 2.24968527e-02 1.01167445e-02
 1.23921826e-02]


# Make Prediction

In [86]:
test_ind['date_recorded'] = pd.to_datetime(test_ind['date_recorded'])
test_ind['construction_year'] = pd.to_datetime(test_ind['construction_year'])

In [96]:
test_ind['duration'] = test_ind['date_recorded'] - test_ind['construction_year']

In [97]:
test_vars = test_ind[['duration', 'water_quality', 'management']]

In [104]:
test_vars.head()

,duration,water_quality,management,duration_seconds
0,15739 days 23:59:59.999997988,soft,parastatal,1.359936e+09
1,15739 days 23:59:59.999998,soft,vwc,1.359936e+09
2,15736 days 23:59:59.999997990,soft,vwc,1.359677e+09
3,15726 days 23:59:59.999998013,soft,vwc,1.358813e+09
4,15790 days 23:59:59.999998,soft,water board,1.364342e+09


In [105]:
test_vars = test_vars.drop('duration', axis=1)

In [127]:
encoded_test_features = encoder.transform(test_vars[categorical_columns])
encoded_test_df = pd.DataFrame(encoded_test_features.toarray(), columns=encoder.categories_[0].tolist() + encoder.categories_[1].tolist())

test_vars_encoded = pd.concat([test_vars.drop(categorical_columns, axis=1), encoded_test_df], axis=1)


In [129]:
predictions = clf.predict(test_vars_encoded)

In [133]:
predictions

array([0, 0, 0, ..., 2, 0, 0])

In [137]:
predictions_submit = label_encoder.inverse_transform(predictions)

In [138]:
predictions_submit

array(['functional', 'functional', 'functional', ..., 'non functional',
       'functional', 'functional'], dtype=object)

In [139]:
submission_df = pd.DataFrame(predictions_submit, columns=['status_group'])

In [140]:
submission_df.to_csv('submission.csv', index=False)